# Andvaranaut tutorial

## Forward Module (Input distributions specified)

### Latin Hypercube Sampling
Import items form forward module as well as utils module

In [ ]:
from andvaranaut.forward import *
from andvaranaut.utils import *

In [ ]:
# Magic features for development purposes
%load_ext autoreload
%autoreload 2

User supplies target function, which takes a 1D numpy array of nx inputs and returns a 1D numpy array of ny outputs. They must also supply a list of univariate distributions from scipy stats for each of the nx inputs.

In [ ]:
# Example target function (2 inputs, 2 outputs)
# A more complex target function will produce an input file, 
# execute external code, and perform post-processing on output
def test_fun(x):
  x1,x2 = x
  return np.array([x1**2+x2,x1**3*x2**2])

# Input variable probability distributions
import scipy.stats as st
sample_space = [st.uniform(loc=0,scale=2),\
                st.norm(loc=1,scale=0.5)]

In [ ]:
# Latin hypercube class instance, with correct arguments
l = LHC(nx=2,ny=2,priors=sample_space,target=test_fun,\
       parallel=False,nproc=1) # Last 2 args are optional & default

Sampling makes use of the latin_random function from py-design

In [ ]:
# Sample input distributions by LHC and evaluate target function
l.sample(nsamps=4)
print(l.x)
print(l.y)

Parallel execution makes use of the ray package. This also works with SLURM submission if a SLURM script calls a python script with these commands in. (Tutorial will be added at a later date)

In [ ]:
# Can also execute target function evaluation in parallel
l.parallel = True
l.nproc = 4
l.sample(nsamps=4)
ray.shutdown() # Shutdown ray parallelism, this command only good for interactive sessions
l.parallel = False
print(l.x)
print(l.y)

In [ ]:
# Plot output distributions based on kernel density estimation
l.y_dist()

In [ ]:
# Optionally delete n samples
# Default is deletion by closest sample to a coarse LHC of number of samples for deletion
l.del_samples(ndels=2,method='coarse_lhc')
print(l.x)
print(l.y,'\n')
# Can also delete by random indexing
l.del_samples(ndels=2,method='random')
print(l.x)
print(l.y,'\n')
# or by specific data indexes
l.del_samples(method='specific',idx=[0,1])
print(l.x)
print(l.y,'\n')

If datasets exist then it is possible to set the class attributes directly with these. They must be in the form of 2d numpy float arrays. An additional consideration is the provided x data must be compatible with the existing distributions. WARNING: This will likely invalidate relationship of samples to selected input distributions and therefore invalidate output distributions.

In [ ]:
x = np.random.rand(5,2)
y = np.random.rand(5,2)
l.set_data(x=x,y=y)
print(l.x)
print(l.y)

### Gaussian process surrogate

In addition to the arguments provided to the lhc class, there are additional arguments for a list of classes which handle conversion and reversion of the x and y datasets, respectively. These are necessary for optimising surrogate performance, and usually consist of transforming bounded ranges on inputs and outputs to unbounded. Normalisations to get numbers O(1) are also useful and can be implemented either here or within the target function.

These conversion/reversion arguments are optional, and can be left blank if desired. Standard methods are provided in andvaranaut.utils, with the logarithm and uniform classes shown below for clarity on the format. A user can define their own class in this format, as long as any additional arguments like the distribution object in uniform are packaged into partial functions within the class.

CHECK NEXT BOX STILL UP TO DATE

In [ ]:
# Convert positive values to unbounded with logarithm
def log_con(y):
  return np.log10(y)
# Revert logarithm with power
def log_rev(y):
  return np.power(10,y)
class logarithm:
  def __init__(self):
    self.con = log_con # Conversion function
    self.rev = log_rev # Reversion function

from functools import partial
# Convert uniform dist samples into standard uniform 
def std_uniform(x,dist):
  intv = dist.interval(1.0)
  x = (x-intv[0])/(intv[1]-intv[0])
  return x
# Revert to original uniform distributions
def uniform_rev(x,dist):
  intv = dist.interval(1.0)
  x = x*(intv[1]-intv[0])+intv[0]
  return x
class uniform:
  def __init__(self,dist):
    self.con = partial(std_uniform,dist=dist)
    self.rev = partial(uniform_rev,dist=dist)

In [ ]:
# Define lists of conversion/reversion class instances for each x and y variable
xconrevs = [uniform(sample_space[0]),normal(sample_space[1])]
yconrevs = [None,logarithm()]
# Instance of gp, only nx, ny, dists, and target are req
g = GP(kernel='RBF',noise=False,xconrevs=xconrevs,yconrevs=yconrevs,\
       nx=2,ny=2,priors=sample_space,target=test_fun,parallel=False,nproc=4)

Methods from lhc class are inherited by the GP class with some additions like automatic conversion of new samples

In [ ]:
g.sample(2)
print(g.x)
print(g.y)
print(g.xc)
print(g.yc)

Can change conversion/reversion classes without reinitialising class

In [ ]:
xconrevs = [logit_logistic(sample_space[0]),None]
yconrevs = [None,nonneg()]
g.change_conrevs(xconrevs,yconrevs)
print(g.x)
print(g.y)
print(g.xc)
print(g.yc)

Take some more samples and fit GP surrogate to converted data

In [ ]:
g.sample(nsamps=98)

In [ ]:
g.fit(restarts=3)

Your fitted model will be a GPy GPRegression object and as such will retain all GPy functionality. It can be accessed as the 'm' attribute of the gp class.

In [ ]:
print(g.m[''])

Make a train-test set and produce plots to assess GP performance

In [ ]:
g.train_test(training_frac=0.9)

In [ ]:
g.test_plots(revert=True,restarts=1,yplots=True,xplots=True)

Can change model details (kernel choice and whether data contains noise) without reinitialsing class. This will scrub any fitted model and require a new call to gp.fit().

In [ ]:
g.change_model(kernel='Exponential',noise=True)
g.fit()
g.test_plots()

Propagate uncertainty using surrogate and get target distributions

In [ ]:
g.change_model(kernel='Exponential',noise=False)
g.fit()
x,y = g.y_dist(mode='hist_kde',nsamps=10000,return_data=True,surrogate=True)

Plot also the distrutions based on underyling 100 LHC samples

In [ ]:
g.y_dist(surrogate=False)

Get 1000 actual function evaluations for comparison with GP surrogate plots. GP based on 100 evaluations gives good qualitative agreement.

In [ ]:
l = LHC(nx=2,ny=2,priors=sample_space,target=test_fun)

In [ ]:
l.sample(1000)

In [ ]:
l.y_dist()

The lengthscales of the fitted GP give an insight into sensitivities of the outputs to the inputs. Relative log importances can be plotted either using converted or original datasets.

In [ ]:
g.relative_importances()
g.relative_importances(original_data=True,restarts=10)

In addition to LHC samples, there is also the option of utilising adaptive sampling. This aims to add samples at points which best balance improved accuracy and input parameter space exploration. Method based on Mohammadi, Hossein, et al. "Cross-validation based adaptive sampling for Gaussian process models." arXiv preprint arXiv:2005.01814 (2020).

In [ ]:
g = GP(kernel='RBF',noise=False,xconrevs=xconrevs,yconrevs=yconrevs,\
       nx=2,ny=2,priors=sample_space,target=test_fun,parallel=False,nproc=4)
g.sample(50)
g.sample(nsamps=5,batchsize=2,method='adaptive')

## Inverse module

In [ ]:
from andvaranaut.inverse import *

### Maximum a posteriori (MAP)

In [ ]:
m = MAP(nx_exp=1,nx_model=1,ny=2,priors=sample_space[1:],\
        target=test_fun)

In [ ]:
xtest = np.random.rand(1,2)
print(xtest)
y = np.array([test_fun(xtest[0])])
m.set_observations(y=y,y_noise=None,\
                   x_exp=xtest[:,0].reshape((1,1)))
print(m.y_obv)
print(m.y_noise)

In [ ]:
m.opt()
print(m.xopt)

### MAP using a GP

In [ ]:
gm = GPMAP(kernel='Matern32',noise=False,nx_exp=1,nx_model=1,ny=2,priors=sample_space,\
        target=test_fun,xconrevs=[uniform(sample_space[0]),normal(sample_space[1])])

In [ ]:
gm.sample(100)

In [ ]:
gm.sample(10,batchsize=1,method='adaptive')

In [ ]:
gm.fit()

In [ ]:
print(gm.m[''])

In [ ]:
xtest = np.random.rand(1,2)
print(xtest)
y = np.array([test_fun(xtest[0])])
gm.set_observations(y=y,y_noise=None,
                   x_exp=xtest[:,0].reshape((1,1)))
print(gm.y_obv)
print(gm.y_noise)

In [ ]:
gm.opt()
print(gm.xopt)

## Utils module

### Save and load objects

In [ ]:
# Save lhc class including datasets
l = LHC(nx=2,ny=2,priors=sample_space,target=test_fun)
l.sample(2)
save_object(obj=l,fname='lhc_tut.pickle')

In [ ]:
# Load lhc class
l = load_object(fname='lhc_tut.pickle')
print(l.x)
print(l.y)

### Standard conversion/reversion classes

Some examples were shown previously in the tutorial but the full list will be given here for completeness:

normal - requires dist argument upon initialisation and converts samples from any normal distribution to a standard normal sample. 
  
uniform - as above but any uniform distribution to standard uniform.  
  
logit-logistic - requires dist argument and converts any uniform distribution sample to an unbounded range via logit  
  
probit - as above but converts to standard normal sample via distribution cdf's  
  
nonneg - converts any non-negative values to unbounded values by the transformation y' = y/(1+y) which has a range [0,1] and then taking the logit.  
  
logarithm - takes a log with base ten to transform any positive values to unbounded  

normalise - requires a fac argument and applies a y/fac conversion